In [ ]:
%env DESI_LOGLEVEL ERROR
from astropy.io import fits
import sys
import glob
import numpy
import webdataset as wds

from desispec.io import read_spectra, write_spectra
from desispec.spectra import Spectra
from desispec.coaddition import coadd_cameras
from astropy.table import Table, Column, vstack, hstack, join

### TARGET bitmasks
It has been confirmed that each spectrum is uniquely `SV1`, `SV2`, `SV3`, or Main Survey.  We thus compress these columns into one.

    desi=> select  count(*) from everest.zpix_redshifts where (SV2_DESI_TARGET !=0 or SV2_MWS_TARGET !=0 or SV2_BGS_TARGET !=0 or SV2_SCND_TARGET !=0) AND (DESI_TARGET !=0 or MWS_TARGET !=0 or BGS_TARGET !=0);
     count 
    -------
         0
    (1 row)

In [ ]:
def getTARGET(fibermap):
    
    targets = ["DESI_TARGET", "MWS_TARGET", "BGS_TARGET", "SCND_TARGET"]
    
    ans=[]
    for target in targets:
        colnames=[dum for dum in fibermap.columns.keys() if target in dum]
        if len(colnames) == 0:
            desi_target = Column(numpy.zeros(len(fibermap)), name=target, dtype=numpy.int64)
        else:
            desi_target = fibermap[colnames[0]]
            dum = fibermap[colnames[0]] !=0
            for i in range(1,len(colnames)):
                desi_target = desi_target+ fibermap[colnames[i]]
                dum = dum + fibermap[colnames[i]] !=0

            if max(dum)>1:
                raise RuntimeError(target+": unexpected behavior as only one of these is expected")

        ans.append(desi_target)

    return ans

## Everest

There are three types of data, `perexp`, `pernight`, and `cumulative`.  Each has its own subdirectory structure.  Note that the contents of the files may be identical depending if there is only one
exposure.

### Spectra
While spectra are saved in different files, the `coadd*.fits` provide a common format for the spectra we want from all three subdirectories.

HDUs inside the fits file are 'FIBERMAP', 'EXP_FIBERMAP', 'X_WAVELENGTH', 'X_FLUX', 'X_IVAR', 'X_MASK', 'X_RESOLUTION', 'SCORES' where X \in B, R, Z

FIBERMAP contains metadata that we want to save.



### Redshifts
Redshift information is in redrock files.

HDUs inside the fits file are 'REDSHIFTS', 'FIBERMAP', 'EXP_FIBERMAP', 'TSNR2'

### SV's
The bitmasks for SV1 are consistent with others so SV1 targets are omitted

In [ ]:
rootdir="/global/project/projectdirs/desi/spectro/redux/everest/tiles/"
subdirs=["perexp", "pernight","cumulative"]
subdir=subdirs[1]
allcoadd_files = glob.glob(rootdir+subdir+'/*/*/coadd-*-*-*.fits')

In [ ]:
sink = wds.TarWriter("../webdataset/test.tar")
index=0

In [ ]:
for name in allcoadd_files:
    if index == 50000:
        print('All Done')
        break   
        
    #scrub path for information
    dum = name.split('/')
    sd1 = dum[-3]
    sd2 = dum[-2]
    subname = name[name.find('coadd')+6:-5]
    dum = subname.split('-')
    panel=dum[0]
    
    zname=name.replace('coadd','redrock')
    
    #Read spectra and coadd cameras, read 
    
    pspectra = read_spectra(name)
    cspectra = coadd_cameras(pspectra)
    zbest = Table.read(zname, 'redshifts')

    #Make selection: only targets in a good fiber

    select = (cspectra.fibermap['OBJTYPE'] == 'TGT') & (cspectra.fibermap['COADD_FIBERSTATUS'] == 0)
    
    # Also select targets not from SV1.  SV1 targets have a non-zero
    # value in a column SV1_*_TARGET
    sv1_colnames=[dum for dum in cspectra.fibermap.columns.keys() if "SV1_" in dum]
    if len(sv1_colnames) !=0:
        notinSV1 = cspectra.fibermap[sv1_colnames[0]] ==0
        for i in range(1,len(sv1_colnames)):
            notinSV1 = numpy.logical_and(notinSV1, cspectra.fibermap[sv1_colnames[i]] ==0)
        select = numpy.logical_and(select,notinSV1)

    # if there is nothing selected move on to the next file
    if select.sum()==0:
        continue

    #Access the relevant keys
    
    # tid = cspectra.fibermap[select]['TARGETID']
    # tra = cspectra.fibermap[select]['TARGET_RA']
    # tdec = cspectra.fibermap[select]['TARGET_DEC']

    desi_target, mws_target, bgs_target, scnd_target = getTARGET(cspectra.fibermap[select])
    
    # zs = zbest[select]['Z'],
    # zerr = zbest[select]['ZERR']
    # zwarn = zbest[select]['ZWARN']
    # spectype = zbest[select]['SPECTYPE']
    # subtype = zbest[select]['SUBTYPE']
    # deltachi2 = zbest[select]['DELTACHI2']
        
    
    # Access spectra here
    
    allwave = cspectra.wave['brz']
    loglam=numpy.log(allwave)
    allflux = cspectra.flux['brz'][select]
    allivar = cspectra.ivar['brz'][select]
    allmask = cspectra.mask['brz'][select]
    
    for flux,ivar,mask,fm, red,d_t, m_t, b_t, s_t in zip(allflux,allivar,allmask,cspectra.fibermap[select], zbest[select], desi_target, mws_target, bgs_target, scnd_target):
        meta=dict()
        meta['Z']=red['Z']
        meta['ZERR']=red['ZERR']
        meta['ZWARN']=red['ZWARN']
        meta['SPECTYPE']=red['SPECTYPE']
        meta['SUBTYPE']=red['SUBTYPE']
        meta['DELTACHI2']=red['DELTACHI2']
        meta['TARGETID']=fm['TARGETID']
        meta['TARGET_RA']=fm['TARGET_RA']
        meta['TARGET_DEC']=fm['TARGET_DEC']
        meta['DESI_TARGET']=d_t
        meta['MWS_TARGET']=m_t
        meta['BGS_TARGET']=b_t
        meta['SCND_TARGET']=s_t
        
        sdata = numpy.array([loglam,flux*allwave,ivar/allwave**2,mask])

        if (index+1)%5000==0:
            print(f"{index:8d}", end="\r", flush=True, file=sys.stderr)
        sink.write({
            "__key__": "sample%08d" % index,
            "spectra.npy": sdata,
            "meta.pyd": meta,
        })
        index=index+1
        
        if index == 50000:
            print('Done')
            break
            


In [ ]:
sink.close()